[![Open in Colab](http://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/psse-cpu/ml-workshop/blob/main/notebooks/workshop-templates/oregon.ipynb)

In [97]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

np.set_printoptions(precision=3, suppress=True)

In [98]:
df = pd.read_csv('https://raw.githubusercontent.com/psse-cpu/ml-workshop/main/notebooks/workshop-templates/oregon-house-prices.csv')
print('All rows of the CSV:\n\n')
df

All rows of the CSV:




,House Size (ft²),Bedrooms,Price
0,2104,3,399900
1,1600,3,329900
2,2400,3,369000
3,1416,2,232000
4,3000,4,539900
5,1985,4,299900
6,1534,3,314900
7,1427,3,198999
8,1380,3,212000
9,1494,3,242500


Non-alphanumeric chars in column names are _"sabad"_/annoying, they're difficult to type, and may
even lead to errors like
```
    House Size (ft² )>=1600 and House Size (ft² )<=1700
                  ^
    SyntaxError: Python keyword not valid identifier in numexpr query
```
so let's rename

In [99]:
df = df.rename(columns={'House Size (ft²)': 'House Size'})
df.head() # show only first few rows, default is 5

,House Size,Bedrooms,Price
0,2104,3,399900
1,1600,3,329900
2,2400,3,369000
3,1416,2,232000
4,3000,4,539900


For the purpose of normalization, let's get the lowest and highest house size, and also bedrooms. 

In [100]:
df.min()

House Size       852
Bedrooms           1
Price         169900
dtype: int64

In [101]:
df.max()

House Size      4478
Bedrooms           5
Price         699900
dtype: int64

While Tensorflow works with _some_ Pandas DataFrames, some libraries like [Scikit-learn](https://scikit-learn.org/), which we'll use for feature scaling here, don't work well with Pandas DataFrames.  Thus, you'll see the `to_numpy()` conversion when extracting
$X$ and $y$.

> Unfortunately, scikit-learn works directly with numpy arrays or scipy sparse arrays, but not `pandas.DataFrame` which is widespread in data science work.

And when not converting, we see this little warning.

```
UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
```

In [102]:
X = df[['House Size', 'Bedrooms']].to_numpy()
y = df[['Price']].to_numpy() / 100_000
print('The first 10 rows as a Numpy array:\n', X[0:10])

The first 10 rows as a Numpy array:
 [[2104    3]
 [1600    3]
 [2400    3]
 [1416    2]
 [3000    4]
 [1985    4]
 [1534    3]
 [1427    3]
 [1380    3]
 [1494    3]]


In [103]:
print('The first 10 prices as a Numpy array:\n', y[0:10])

The first 10 prices as a Numpy array:
 [[3.999]
 [3.299]
 [3.69 ]
 [2.32 ]
 [5.399]
 [2.999]
 [3.149]
 [1.99 ]
 [2.12 ]
 [2.425]]


### Wait a minute, why divide price by 10,000?

Because our loss function is Mean Square Error,

it will get the **sum** of the **SQUARES** of very large numbers first, before getting the average
by dividing it with the number of training samples $m$, as seen in the formula:

#### Vectorized
$\frac{1}{2m}(\hat{y} - y)^T(\hat{y} - y)$

### Unvectorized
$\frac{1}{2m} \sum\limits_{i = 1}^{m}(y_i - \hat{y_i})^2$

And if we don't do it, our first very very wrong attempt will lead to very large sums,
and thus very very large - like $\infty$ LARGE - loss.


```
Epoch 1/200
2/2 [==============================] - 0s 2ms/step - loss: 3346001899810717696.0000
Epoch 2/200
2/2 [==============================] - 0s 3ms/step - loss: 17488194481918713621838967942938624.0000
Epoch 3/200
2/2 [==============================] - 0s 2ms/step - loss: inf
Epoch 4/200
2/2 [==============================] - 0s 3ms/step - loss: inf
Epoch 5/200
2/2 [==============================] - 0s 2ms/step - loss: inf
```

As discussed in the slides, we do feature scaling first.

In [104]:
scaler = MinMaxScaler().fit(X)
X_norm = scaler.transform(X)
print('Original X:\n', X[0:10], '\n\n', 'Normalized X:\n', X_norm[0:10])

Original X:
 [[2104    3]
 [1600    3]
 [2400    3]
 [1416    2]
 [3000    4]
 [1985    4]
 [1534    3]
 [1427    3]
 [1380    3]
 [1494    3]] 

 Normalized X:
 [[0.345 0.5  ]
 [0.206 0.5  ]
 [0.427 0.5  ]
 [0.156 0.25 ]
 [0.592 0.75 ]
 [0.312 0.75 ]
 [0.188 0.5  ]
 [0.159 0.5  ]
 [0.146 0.5  ]
 [0.177 0.5  ]]


## Target values for $w$

I cross-checked this in another _"less-overkill"_ library - Scikit-learn, using its
`sklearn.linear_model.LinearRegression` model, and I got the following results:

$$
w = \begin{bmatrix}
  w_0 \\
  w_1 \\
  w_2
\end{bmatrix} = 
\begin{bmatrix}
  1.995 \\
  5.048 \\
  -0.35
\end{bmatrix}
$$

If we want to predict the price of a $1,650$ sq.ft. house with $3$ bedrooms, then
we have to normalize these inputs as well. 
- $1,650$ becomes $0.22$, and
- $3$ being the median becomes $0.5$.

So that's 

$\hat{y} = w_2x_2 + w_1x_1 + w_0$  
$\hat{y} = -0.35 \cdot 0.5 + 5.048 \cdot 0.22 + 1.995$  
$\hat{y} = 2.93056$

but we have to scale it back to $\times 100k$ USD, our prediction is $293,056$ USD.

So the model you will code below in Tensorflow should be somewhere around that ballpark.

**⚠ Spoiler alert:** I got:
$$
w = \begin{bmatrix}
  w_0 \\
  w_1 \\
  w_2
\end{bmatrix} = 
\begin{bmatrix}
  1.96 \\
  4.975 \\
  -0.26
\end{bmatrix}
$$

- it predicts $291,623.855$ for the same house size ($1650$) and # of bedrooms ($3$).
- it used $\alpha = 0.05$, and trained for $1,000$ epochs.

Let's see if $291.6k$ USD is a good prediction for a $1,650 ft^2$ house with $3$ bedrooms.

In [105]:
df.query('`House Size` >= 1500 and `House Size` <= 2000').sort_values('House Size')

,House Size,Bedrooms,Price
6,1534,3,314900
1,1600,3,329900
22,1604,3,242900
40,1664,2,368500
20,1767,3,252900
34,1811,4,285900
30,1839,2,349900
45,1852,4,299900
21,1888,2,255000
12,1890,3,329999


Right, $1,650$ is priced at $291k$, which sorta make sense, slightly more expensive than the one
$1,604$ sq.ft. big.  

You'll also see a smaller house being more expensive, and a bigger house that's cheaper, but 🤷,
we don't have lots of _"training examples"_ to begin with, at only $47$ houses.

# Your turn

take it from here, you already have these matrices and vectors:

```py
# X_norm  - our normalized X, a 47x2 matrix
# y       - our prices in the training set, a 47x1 vector
```


In [106]:
# declare, compile, and train your model here

# <YOUR CODE HERE>

In [109]:
# predict the price of a house 1,650 sq.ft. in size with 3 bedrooms
medium_sized_house = np.array([
  [1650, 3]
])

scaled_input = scaler.transform(medium_sized_house)
scaled_input

array([[0.22, 0.5 ]])

In [111]:
# <YOUR PREDICTION CODE HERE>